In [1]:
import ampligraph
import numpy as np
import pandas as pd
import pickle
import tensorflow as tf
from ampligraph.evaluation import train_test_split_no_unseen, evaluate_performance, mr_score, mrr_score, hits_at_n_score
from ampligraph.discovery import query_topn, discover_facts, find_clusters
from ampligraph.latent_features import TransE, ComplEx, HolE, DistMult, ConvE, ConvKB
from ampligraph.utils import save_model, restore_model

def display_aggregate_metrics(ranks):
    print('Mean Rank:', mr_score(ranks)) 
    print('Mean Reciprocal Rank:', mrr_score(ranks)) 
    print('Hits@1:', hits_at_n_score(ranks, 1))
    print('Hits@10:', hits_at_n_score(ranks, 10))
    print('Hits@100:', hits_at_n_score(ranks, 100))

print('Ampligraph version: {}'.format(ampligraph.__version__))

Ampligraph version: 1.4.0


# 1 导入数据集

In [2]:
with open('../data/drug_target_triplets.pkl', 'rb') as f:
    target_df = pickle.load(f)

In [3]:
with open('../data/drug_transporter_triplets.pkl', 'rb') as f:
    transporter_df = pickle.load(f)

In [4]:
with open('../data/drug_enzyme_triplets.pkl', 'rb') as f:
    enzyme_df = pickle.load(f)

In [5]:
with open('../data/drug_indication_triplets.pkl', 'rb') as f:
    indication_df = pickle.load(f)

In [6]:
with open('../data/drug_sideeffect_triplets.pkl', 'rb') as f:
    sideeffect_df = pickle.load(f)

In [7]:
# 拼接所有三元组
all_triplets = pd.concat([target_df, transporter_df, enzyme_df, 
                         indication_df, sideeffect_df], axis=0, ignore_index=True)

all_triplets

subject        relation    object
0       DB00001      has_target    P00734
1       DB00002      has_target    P00533
2       DB00002      has_target    O75015
3       DB00002      has_target    P02745
4       DB00002      has_target    P02746
...         ...             ...       ...
136835  DB15263  has_sideeffect  C0376288
136836  DB15263  has_sideeffect  C0541799
136837  DB15263  has_sideeffect  C1268766
136838  DB15263  has_sideeffect  C2225524
136839  DB15263  has_sideeffect  C2242996

[136840 rows x 3 columns]

In [8]:
# 所有三元组中主语的类别数
len(set(all_triplets.subject))

7908

In [9]:
# 所有三元组中谓语的类别数
len(set(all_triplets.object))

10922

In [10]:
# 拼接部分三元组
triplets_4 = pd.concat([target_df, indication_df, sideeffect_df], axis=0, ignore_index=True)

triplets_4

subject        relation    object
0       DB00001      has_target    P00734
1       DB00002      has_target    P00533
2       DB00002      has_target    O75015
3       DB00002      has_target    P02745
4       DB00002      has_target    P02746
...         ...             ...       ...
128597  DB15263  has_sideeffect  C0376288
128598  DB15263  has_sideeffect  C0541799
128599  DB15263  has_sideeffect  C1268766
128600  DB15263  has_sideeffect  C2225524
128601  DB15263  has_sideeffect  C2242996

[128602 rows x 3 columns]

In [11]:
# 所有三元组中主语的类别数
len(set(triplets_4.subject))

7582

In [12]:
# 所有三元组中谓语的类别数
len(set(triplets_4.object))

10645

# 1 划分分类模型的训练集和测试集
- drug_indication_triplets作为负样本
- drug_sideeffect_triplets作为正样本
- 在确保所有实体均被embedding的情况下,还需要确保训练样本对应的三元组均被embedding,而测试样本对应的三元组均没有被embedding

## 1.1 分类模型训练集和测试集
- 按9:1划分正负样本训练集和测试集

In [13]:
# 负样本总数
len(indication_df)

10145

In [14]:
# 取10%的样本数量
len(indication_df)*0.1

1014.5

### (1)负样本测试集

- 从至少出现3次的indication中随机抽取1014个样本作为测试集,避免部分drug和indication没有被embedding

In [15]:
# 对indication出现的次数进行统计
indication_count = indication_df.object.value_counts()
indication_test = indication_count[indication_count.values >= 3]
negative_test = indication_df[indication_df.object.isin(indication_test.index)].sample(n=1014, random_state=42)
print("适应症至少与3种药物存在链接关系的三元组个数为：{}".format(\
                                          len(indication_df[indication_df.object.isin(indication_test.index)])))

适应症至少与3种药物存在链接关系的三元组个数为：8364


In [16]:
negative_test

subject        relation    object
2936   DB00513  has_indication  C0023890
8330   DB01320  has_indication  C0011206
8045   DB01241  has_indication  C0030305
7062   DB01141  has_indication  C0239295
6595   DB01059  has_indication  C0033581
...        ...             ...       ...
10011  DB13841  has_indication  C0011253
4531   DB00741  has_indication  C0004096
578    DB00196  has_indication  C0239295
5672   DB00928  has_indication  C0026986
4719   DB00755  has_indication  C0007114

[1014 rows x 3 columns]

### (2)负样本训练集
- 排除负样本测试集之外的样本作为分类器负样本的训练集

In [17]:
negative_train = indication_df[~indication_df.index.isin(negative_test.index)]

In [18]:
negative_train

subject        relation    object
0      DB00001  has_indication  C0002965
1      DB00001  has_indication  C0040034
2      DB00001  has_indication  C0040038
3      DB00001  has_indication  C0155626
4      DB00001  has_indication  C0272285
...        ...             ...       ...
10138  DB14740  has_indication  C0032797
10140  DB00781  has_indication  C0009450
10141  DB00781  has_indication  C0042029
10142  DB15263  has_indication  C0032019
10144  DB15263  has_indication  C0700502

[9131 rows x 3 columns]

### (3)正样本测试集
- 数量需与负样本测试集相等,避免样本不平衡问题
- 从至少出现3次的sideeffect中随机抽取1014个样本作为测试集,避免部分drug和sideeffect没有被embedding

In [15]:
# 对sideeffect出现的次数进行统计
sideeffect_count = sideeffect_df.object.value_counts()
sideeffect_test = sideeffect_count[sideeffect_count.values >= 3]
positive_test = sideeffect_df[sideeffect_df.object.isin(sideeffect_test.index)].sample(n=1014, random_state=42)

In [20]:
positive_test

subject        relation    object
90681  DB06282  has_sideeffect  C0600142
75629  DB01217  has_sideeffect  C0042109
67055  DB01101  has_sideeffect  C0014335
16617  DB00313  has_sideeffect  C0012569
17066  DB00315  has_sideeffect  C0424295
...        ...             ...       ...
47954  DB00776  has_sideeffect  C0043352
22028  DB00401  has_sideeffect  C0423791
79916  DB01259  has_sideeffect  C0030252
30762  DB00526  has_sideeffect  C0178298
63773  DB01037  has_sideeffect  C0231218

[1014 rows x 3 columns]

### (4)正样本训练集
- 排除正样本测试集之外的样本作为分类器正样本的训练集

In [21]:
positive_train = sideeffect_df[~sideeffect_df.index.isin(positive_test.index)]

In [22]:
positive_train

subject        relation    object
0       DB00001  has_sideeffect  C0000833
1       DB00001  has_sideeffect  C0002792
2       DB00001  has_sideeffect  C0002871
3       DB00001  has_sideeffect  C0002994
4       DB00001  has_sideeffect  C0006266
...         ...             ...       ...
100298  DB15263  has_sideeffect  C0376288
100299  DB15263  has_sideeffect  C0541799
100300  DB15263  has_sideeffect  C1268766
100301  DB15263  has_sideeffect  C2225524
100302  DB15263  has_sideeffect  C2242996

[99289 rows x 3 columns]

# 2 检验用于embedding的三元组集合中是否存在实体遗漏
- 即正负样本的训练集是否均包含了所有的实体
- 若实体数量不存在遗漏,则可以进行embedding

## 2.1 6种节点的三元组

- subject=7908
- object=10922

In [23]:
# 6种节点的三元组
embedding_triplets_6 = pd.concat([target_df, transporter_df, enzyme_df, 
                         negative_train, positive_train], axis=0, ignore_index=True)

In [24]:
# 所有三元组中主语的类别数
len(set(embedding_triplets_6.subject))

7908

In [25]:
# 所有三元组中谓语的类别数
len(set(embedding_triplets_6.object))

10922

In [26]:
embedding_triplets_6

subject        relation    object
0       DB00001      has_target    P00734
1       DB00002      has_target    P00533
2       DB00002      has_target    O75015
3       DB00002      has_target    P02745
4       DB00002      has_target    P02746
...         ...             ...       ...
134807  DB15263  has_sideeffect  C0376288
134808  DB15263  has_sideeffect  C0541799
134809  DB15263  has_sideeffect  C1268766
134810  DB15263  has_sideeffect  C2225524
134811  DB15263  has_sideeffect  C2242996

[134812 rows x 3 columns]

## 2.2 4种节点的三元组

- subject = 7582
- object = 10645

In [27]:
# 4种节点的三元组
embedding_triplets_4 = pd.concat([target_df, negative_train, positive_train], axis=0, ignore_index=True)

In [28]:
embedding_triplets_4

subject        relation    object
0       DB00001      has_target    P00734
1       DB00002      has_target    P00533
2       DB00002      has_target    O75015
3       DB00002      has_target    P02745
4       DB00002      has_target    P02746
...         ...             ...       ...
126569  DB15263  has_sideeffect  C0376288
126570  DB15263  has_sideeffect  C0541799
126571  DB15263  has_sideeffect  C1268766
126572  DB15263  has_sideeffect  C2225524
126573  DB15263  has_sideeffect  C2242996

[126574 rows x 3 columns]

In [29]:
# 所有三元组中主语的类别数
len(set(embedding_triplets_4.subject))

7582

In [30]:
# 所有三元组中谓语的类别数
len(set(embedding_triplets_4.object))

10645

# 3 整合分类器训练集和测试集
- num of negative_train = 9131
- num of postive_train = 99289
- 为避免训练集中由于正负样本数量不平衡造成训练效果较差,这里将训练集的负样本数量*10 (=91310) 进行处理

In [31]:
df=pd.DataFrame()
 
for i in range(len(negative_train)):
    row_data = negative_train.iloc[i,:]
    sample = pd.DataFrame([row_data]*10)     #每行复制10倍
    df = df.append(sample)

In [32]:
negative_train = df
negative_train

subject        relation    object
0      DB00001  has_indication  C0002965
0      DB00001  has_indication  C0002965
0      DB00001  has_indication  C0002965
0      DB00001  has_indication  C0002965
0      DB00001  has_indication  C0002965
...        ...             ...       ...
10144  DB15263  has_indication  C0700502
10144  DB15263  has_indication  C0700502
10144  DB15263  has_indication  C0700502
10144  DB15263  has_indication  C0700502
10144  DB15263  has_indication  C0700502

[91310 rows x 3 columns]

### (1)训练集
- negative_train + positive_train

In [33]:
negative_train = negative_train.drop('relation', axis=1)
negative_train.insert(2, 'label', [0 for i in range(len(negative_train))])

positive_train = positive_train.drop('relation', axis=1)
positive_train.insert(2, 'label', [1 for i in range(len(positive_train))])

train_dataset = pd.concat([negative_train, positive_train],axis=0, ignore_index=True)

In [34]:
train_dataset

subject    object  label
0       DB00001  C0002965      0
1       DB00001  C0002965      0
2       DB00001  C0002965      0
3       DB00001  C0002965      0
4       DB00001  C0002965      0
...         ...       ...    ...
190594  DB15263  C0376288      1
190595  DB15263  C0541799      1
190596  DB15263  C1268766      1
190597  DB15263  C2225524      1
190598  DB15263  C2242996      1

[190599 rows x 3 columns]

### (2)测试集
- negative_test + positive_test

In [35]:
negative_test = negative_test.drop('relation', axis=1)
negative_test.insert(2, 'label', [0 for i in range(len(negative_test))])

positive_test = positive_test.drop('relation', axis=1)
positive_test.insert(2, 'label', [1 for i in range(len(positive_test))])

test_dataset = pd.concat([negative_test, positive_test],axis=0, ignore_index=True)

In [36]:
test_dataset

subject    object  label
0     DB00513  C0023890      0
1     DB01320  C0011206      0
2     DB01241  C0030305      0
3     DB01141  C0239295      0
4     DB01059  C0033581      0
...       ...       ...    ...
2023  DB00776  C0043352      1
2024  DB00401  C0423791      1
2025  DB01259  C0030252      1
2026  DB00526  C0178298      1
2027  DB01037  C0231218      1

[2028 rows x 3 columns]

# 4 知识图谱(KG) embedding

## 4.1 划分三元组embedding的训练集、验证集、测试集

In [37]:
# 4种节点
dataset = embedding_triplets_4

# 6种节点
# dataset = embedding_triplets_6

In [38]:
from ampligraph.evaluation import train_test_split_no_unseen

# get the validation set of size 500
# 获取大小为 500 的验证集
test_train, X_valid = train_test_split_no_unseen(dataset.values, 500, seed=0)

# get the test set of size 1000 from the remaining triples
# 从剩余的三元组中获取大小为 1000 的测试集
X_train, X_test = train_test_split_no_unseen(test_train, 1000, seed=0)

print('Total triples:', dataset.shape)
print('Size of train:', X_train.shape)
print('Size of valid:', X_valid.shape)
print('Size of test:', X_test.shape)

Total triples: (126574, 3)
Size of train: (125074, 3)
Size of valid: (500, 3)
Size of test: (1000, 3)


- 定义保存嵌入模型及嵌入向量的函数

In [39]:
# 保存嵌入模型以及实体、关系的嵌入向量
from ampligraph.utils import save_model, restore_model
from ampligraph.utils import create_tensorboard_visualizations

# 保存嵌入模型
def save_embedding_model(model, filepath):
    save_model(model, filepath)
    print('finish!')

# 保存嵌入向量
def save_embedding_vector(model, filepath):
    create_tensorboard_visualizations(model, filepath)
    print('finish!')

## 4.2 DisMult

### 4.2.1 训练嵌入模型

In [ ]:
# print('Training set:', X_train.shape)

# model = DistMult(k=300, epochs=300, eta=10, loss='multiclass_nll', 
#                 initializer='xavier', initializer_params={'uniform': False},
#                 regularizer='LP', regularizer_params= {'lambda': 0.0001, 'p': 3},
#                 optimizer= 'adam', optimizer_params= {'lr': 0.001}, 
#                 seed= 0, batches_count= 1, verbose=True)

# model.fit(X_train)

# # create the filter （创建过滤器）
# X_filter = np.concatenate([X_train, X_valid, X_test], 0)

# ranks = evaluate_performance(X_test, 
#                              model=model,
#                              filter_triples=X_filter,
#                              corrupt_side='s,o',
#                              ranking_strategy='worst')

# # ranks are computed per triple （每三元组计算排名）
# print('Test set:', X_test.shape)
# print('Size of ranks:', ranks.shape)

# display_aggregate_metrics(ranks)

In [40]:
# 使用默认参数进行训练

def embedding_by_DistMult(X_train, X_valid, X_test, k):
    print('Training set:', X_train.shape)

    model = DistMult(k=k)

    model.fit(X_train)

    # create the filter （创建过滤器）
    X_filter = np.concatenate([X_train, X_valid, X_test], 0)

    ranks = evaluate_performance(X_test, 
                                 model=model,
                                 filter_triples=X_filter,
                                 corrupt_side='s,o',
                                 ranking_strategy='worst')

    # ranks are computed per triple （每三元组计算排名）
    print('Test set:', X_test.shape)
    print('Size of ranks:', ranks.shape)

    display_aggregate_metrics(ranks)
    
    return model

#### (1-1)k=10
- 2x5

In [41]:
# 训练并评估嵌入模型
distmult4_10 = embedding_by_DistMult(X_train, X_valid, X_test, k=10)

Training set: (125074, 3)
Test set: (1000, 3)
Size of ranks: (1000, 2)
Mean Rank: 789.714
Mean Reciprocal Rank: 0.07720416754203817
Hits@1: 0.0385
Hits@10: 0.1505
Hits@100: 0.459


In [42]:
# 保存嵌入模型及嵌入向量
vector_filepath = '../data/dp_embedding_model/DistMult/distmult4_10'
model_filepath = '../data/dp_embedding_model/DistMult/distmult4_10/distmult4_10.pkl'

save_embedding_vector(distmult4_10, vector_filepath)
save_embedding_model(distmult4_10, model_filepath)

finish!
finish!


#### (1-2)k=20
- 4x5

In [43]:
# 训练并评估嵌入模型
distmult4_20 = embedding_by_DistMult(X_train, X_valid, X_test, k=20)

Training set: (125074, 3)
Test set: (1000, 3)
Size of ranks: (1000, 2)
Mean Rank: 659.98
Mean Reciprocal Rank: 0.08497697296490372
Hits@1: 0.038
Hits@10: 0.1725
Hits@100: 0.496


In [44]:
# 保存嵌入模型及嵌入向量
vector_filepath = '../data/dp_embedding_model/DistMult/distmult4_20'
model_filepath = '../data/dp_embedding_model/DistMult/distmult4_20/distmult4_20.pkl'

save_embedding_vector(distmult4_20, vector_filepath)
save_embedding_model(distmult4_20, model_filepath)

finish!
finish!


#### (1) k=36
- 6x6

In [726]:
# 训练并评估嵌入模型
distmult4_36 = embedding_by_DistMult(X_train, X_valid, X_test, k=36)

Training set: (125074, 3)
Test set: (1000, 3)
Size of ranks: (1000, 2)
Mean Rank: 609.307
Mean Reciprocal Rank: 0.08366072867174376
Hits@1: 0.034
Hits@10: 0.185
Hits@100: 0.505


In [727]:
distmult4_36.all_params

{'k': 18,
 'eta': 2,
 'epochs': 100,
 'batches_count': 100,
 'seed': 0,
 'embedding_model_params': {'normalize_ent_emb': False,
  'negative_corruption_entities': 'all',
  'corrupt_sides': ['s,o']},
 'optimizer': 'adam',
 'optimizer_params': {'lr': 0.0005},
 'loss': 'nll',
 'loss_params': {},
 'regularizer': None,
 'regularizer_params': {},
 'initializer': 'xavier',
 'initializer_params': {'uniform': False},
 'verbose': False}

In [58]:
# 保存嵌入模型及嵌入向量
vector_filepath = '../data/dp_embedding_model/DistMult/distmult4_36'
model_filepath = '../data/dp_embedding_model/DistMult/distmult4_36/distmult4_36.pkl'

save_embedding_vector(distmult4_36, vector_filepath)
save_embedding_model(distmult4_36, model_filepath)

finish!
finish!


#### (2) k=64
- 8x8

In [52]:
# 训练并评估嵌入模型
distmult4_64 = embedding_by_DistMult(X_train, X_valid, X_test, k=64)

Training set: (125074, 3)
Test set: (1000, 3)
Size of ranks: (1000, 2)
Mean Rank: 617.8435
Mean Reciprocal Rank: 0.09918096189349279
Hits@1: 0.044
Hits@10: 0.2055
Hits@100: 0.5735


In [55]:
distmult4_64.all_params

{'k': 64,
 'eta': 2,
 'epochs': 100,
 'batches_count': 100,
 'seed': 0,
 'embedding_model_params': {'normalize_ent_emb': False,
  'negative_corruption_entities': 'all',
  'corrupt_sides': ['s,o']},
 'optimizer': 'adam',
 'optimizer_params': {'lr': 0.0005},
 'loss': 'nll',
 'loss_params': {},
 'regularizer': None,
 'regularizer_params': {},
 'initializer': 'xavier',
 'initializer_params': {'uniform': False},
 'verbose': False}

In [59]:
# 保存嵌入模型及嵌入向量
vector_filepath = '../data/dp_embedding_model/DistMult/distmult4_64'
model_filepath = '../data/dp_embedding_model/DistMult/distmult4_64/distmult4_64.pkl'

save_embedding_vector(distmult4_64, vector_filepath)
save_embedding_model(distmult4_64, model_filepath)

finish!
finish!


#### （3） k=121
- 11x11

In [94]:
# 训练并评估嵌入模型
distmult4_121 = embedding_by_DistMult(X_train, X_valid, X_test, k=121)

Training set: (125074, 3)
Test set: (1000, 3)
Size of ranks: (1000, 2)
Mean Rank: 543.755
Mean Reciprocal Rank: 0.11484985837009092
Hits@1: 0.055
Hits@10: 0.225
Hits@100: 0.631


In [95]:
# 保存嵌入模型及嵌入向量
vector_filepath = '../data/dp_embedding_model/DistMult/distmult4_121'
model_filepath = '../data/dp_embedding_model/DistMult/distmult4_121/distmult4_121.pkl'

save_embedding_vector(distmult4_121, vector_filepath)
save_embedding_model(distmult4_121, model_filepath)

finish!
finish!


#### （3-1） k=128
- 8x16

In [63]:
# 训练并评估嵌入模型
distmult4_128 = embedding_by_DistMult(X_train, X_valid, X_test, k=128)

Training set: (125074, 3)
Test set: (1000, 3)
Size of ranks: (1000, 2)
Mean Rank: 578.875
Mean Reciprocal Rank: 0.12325893295706049
Hits@1: 0.064
Hits@10: 0.236
Hits@100: 0.6305


In [64]:
# 保存嵌入模型及嵌入向量
vector_filepath = '../data/dp_embedding_model/DistMult/distmult4_128'
model_filepath = '../data/dp_embedding_model/DistMult/distmult4_128/distmult4_128.pkl'

save_embedding_vector(distmult4_128, vector_filepath)
save_embedding_model(distmult4_128, model_filepath)

finish!
finish!


#### （3-2） k=196
- 14x14

In [42]:
# 训练并评估嵌入模型
distmult4_196 = embedding_by_DistMult(X_train, X_valid, X_test, k=196)

Training set: (125074, 3)
Test set: (1000, 3)
Size of ranks: (1000, 2)
Mean Rank: 571.7575
Mean Reciprocal Rank: 0.1276872148675077
Hits@1: 0.0645
Hits@10: 0.246
Hits@100: 0.648


In [49]:
# 保存嵌入模型及嵌入向量
vector_filepath = '../data/dp_embedding_model/DistMult/distmult4_196'
model_filepath = '../data/dp_embedding_model/DistMult/distmult4_196/distmult4_196.pkl'

save_embedding_vector(distmult4_196, vector_filepath)
save_embedding_model(distmult4_196, model_filepath)

finish!
finish!


#### （4）k=256
- 16x16

In [114]:
# 训练并评估嵌入模型
distmult4_256 = embedding_by_DistMult(X_train, X_valid, X_test, k=256)

Training set: (125074, 3)
Test set: (1000, 3)
Size of ranks: (1000, 2)
Mean Rank: 607.2415
Mean Reciprocal Rank: 0.1319455164541736
Hits@1: 0.0695
Hits@10: 0.2535
Hits@100: 0.6635


In [115]:
# 保存嵌入模型及嵌入向量
vector_filepath = '../data/dp_embedding_model/DistMult/distmult4_256'
model_filepath = '../data/dp_embedding_model/DistMult/distmult4_256/distmult4_256.pkl'

save_embedding_vector(distmult4_256, vector_filepath)
save_embedding_model(distmult4_256, model_filepath)

finish!
finish!


#### （4）K=324
- 18x18

In [133]:
# 训练并评估嵌入模型
distmult4_324 = embedding_by_DistMult(X_train, X_valid, X_test, k=324)

Training set: (125074, 3)
Test set: (1000, 3)
Size of ranks: (1000, 2)
Mean Rank: 584.855
Mean Reciprocal Rank: 0.12950306861553137
Hits@1: 0.0605
Hits@10: 0.2665
Hits@100: 0.675


In [134]:
# 保存嵌入模型及嵌入向量
vector_filepath = '../data/dp_embedding_model/DistMult/distmult4_324'
model_filepath = '../data/dp_embedding_model/DistMult/distmult4_324/distmult4_324.pkl'

save_embedding_vector(distmult4_324, vector_filepath)
save_embedding_model(distmult4_324, model_filepath)

finish!
finish!


#### （5）400
- 20x20

In [149]:
# 训练并评估嵌入模型
distmult4_400 = embedding_by_DistMult(X_train, X_valid, X_test, k=400)

Training set: (125074, 3)
Test set: (1000, 3)
Size of ranks: (1000, 2)
Mean Rank: 616.592
Mean Reciprocal Rank: 0.1354378195262205
Hits@1: 0.071
Hits@10: 0.2645
Hits@100: 0.6825


In [150]:
# 保存嵌入模型及嵌入向量
vector_filepath = '../data/dp_embedding_model/DistMult/distmult4_400'
model_filepath = '../data/dp_embedding_model/DistMult/distmult4_400/distmult4_400.pkl'

save_embedding_vector(distmult4_400, vector_filepath)
save_embedding_model(distmult4_400, model_filepath)

finish!
finish!


#### （6）k=500
- 20x25

In [165]:
# 训练并评估嵌入模型
distmult4_500 = embedding_by_DistMult(X_train, X_valid, X_test, k=500)

Training set: (125074, 3)
Test set: (1000, 3)
Size of ranks: (1000, 2)
Mean Rank: 651.9935
Mean Reciprocal Rank: 0.13697832881501198
Hits@1: 0.0675
Hits@10: 0.285
Hits@100: 0.683


In [166]:
# 保存嵌入模型及嵌入向量
vector_filepath = '../data/dp_embedding_model/DistMult/distmult4_500'
model_filepath = '../data/dp_embedding_model/DistMult/distmult4_500/distmult4_500.pkl'

save_embedding_vector(distmult4_500, vector_filepath)
save_embedding_model(distmult4_500, model_filepath)

finish!
finish!


#### （7）k=600
- 20x30

In [181]:
# 训练并评估嵌入模型
distmult4_600 = embedding_by_DistMult(X_train, X_valid, X_test, k=600)

Training set: (125074, 3)
Test set: (1000, 3)
Size of ranks: (1000, 2)
Mean Rank: 679.103
Mean Reciprocal Rank: 0.1448452001317512
Hits@1: 0.077
Hits@10: 0.284
Hits@100: 0.6915


In [182]:
# 保存嵌入模型及嵌入向量
vector_filepath = '../data/dp_embedding_model/DistMult/distmult4_600'
model_filepath = '../data/dp_embedding_model/DistMult/distmult4_600/distmult4_600.pkl'

save_embedding_vector(distmult4_600, vector_filepath)
save_embedding_model(distmult4_600, model_filepath)

finish!
finish!


#### （8）k=700
- 20x35

In [616]:
# 训练并评估嵌入模型
distmult4_700 = embedding_by_DistMult(X_train, X_valid, X_test, k=700)

Training set: (125074, 3)
Test set: (1000, 3)
Size of ranks: (1000, 2)
Mean Rank: 653.654
Mean Reciprocal Rank: 0.14400313334670764
Hits@1: 0.0775
Hits@10: 0.272
Hits@100: 0.6915


In [617]:
# 保存嵌入模型及嵌入向量
vector_filepath = '../data/dp_embedding_model/DistMult/distmult4_700'
model_filepath = '../data/dp_embedding_model/DistMult/distmult4_700/distmult4_700.pkl'

save_embedding_vector(distmult4_700, vector_filepath)
save_embedding_model(distmult4_700, model_filepath)

finish!
finish!


#### （9）k=800
- 20x40

In [636]:
# 训练并评估嵌入模型
distmult4_800 = embedding_by_DistMult(X_train, X_valid, X_test, k=800)

Training set: (125074, 3)
Test set: (1000, 3)
Size of ranks: (1000, 2)
Mean Rank: 645.4975
Mean Reciprocal Rank: 0.1471585101403478
Hits@1: 0.0775
Hits@10: 0.287
Hits@100: 0.6935


In [637]:
# 保存嵌入模型及嵌入向量
vector_filepath = '../data/dp_embedding_model/DistMult/distmult4_800'
model_filepath = '../data/dp_embedding_model/DistMult/distmult4_800/distmult4_800.pkl'

save_embedding_vector(distmult4_800, vector_filepath)
save_embedding_model(distmult4_800, model_filepath)

finish!
finish!


#### （10）k=900
- 30x30

In [653]:
# 训练并评估嵌入模型
distmult4_900 = embedding_by_DistMult(X_train, X_valid, X_test, k=900)

Training set: (125074, 3)
Test set: (1000, 3)
Size of ranks: (1000, 2)
Mean Rank: 641.789
Mean Reciprocal Rank: 0.1419644094786068
Hits@1: 0.0735
Hits@10: 0.2755
Hits@100: 0.6835


In [654]:
# 保存嵌入模型及嵌入向量
vector_filepath = '../data/dp_embedding_model/DistMult/distmult4_900'
model_filepath = '../data/dp_embedding_model/DistMult/distmult4_900/distmult4_900.pkl'

save_embedding_vector(distmult4_900, vector_filepath)
save_embedding_model(distmult4_900, model_filepath)

finish!
finish!


- 网格搜索

In [229]:
# from ampligraph.evaluation import select_best_model_ranking

# model_class = DistMult
# param_grid = {
#                      "batches_count": [5],
#                      "seed": 0,
#                      "epochs": [20, 50, 100],
#                      "k": [600],
#                      "eta": [1, 3, 5],
#                      "loss": ["multiclass_nll"],
#                      "loss_params": {},
#                      "embedding_model_params": {},
#                      "regularizer": ["LP"],
#                      "regularizer_params": {
#                          "p": [3],
#                          "lambda": [1e-3]
#                       },
#                      "optimizer": ["adam"],
#                      "optimizer_params":{
#                          "lr": 0.001 #lambda: np.random.uniform(0.00001, 0.01)
#                      },
#                      "verbose": False
#                  }
# best_model, best_params, best_mrr_train, ranks_test, mrr_test, experimental_history = \
#         select_best_model_ranking(model_class, 
#                           X_train, 
#                           X_valid, 
#                           X_test, 
#                           param_grid,
#                           # max_combinations=2, # performs random search-executes 2 models by randomly choosing params
#                           use_filter=True, 
#                           verbose=True,
#                           early_stopping=True)

9it [30:28, 203.19s/it]


In [230]:
# print('MRR of the best model:', best_mrr_train)

MRR of the best model: 0.14204611220236513


In [231]:
# params of the best model
best_params

{'batches_count': 5,
 'seed': 0,
 'epochs': 100,
 'k': 600,
 'eta': 5,
 'loss': 'multiclass_nll',
 'regularizer': 'LP',
 'optimizer': 'adam',
 'verbose': False,
 'regularizer_params': {'p': 3, 'lambda': 0.001},
 'optimizer_params': {'lr': 0.001}}

In [232]:
# best_model

### 4.2.2 保存模型及嵌入向量集

In [233]:
# # 保存实体、关系嵌入向量集
# from ampligraph.utils import create_tensorboard_visualizations
# # # 加载模型
# # model = restore_model('../data/DisMult/dismult4_400/dismult4_400.pkl')

# create_tensorboard_visualizations(best_model, '../data/DisMult/dismult4_600')

In [234]:
# # 保存嵌入模型
# from ampligraph.utils import save_model, restore_model
# save_model(best_model, '../data/DisMult/dismult4_600/dismult4_600.pkl')

## 4.3 ComplEx

### 4.3.1 训练嵌入模型 

In [82]:
# 使用默认参数进行训练

def embedding_by_ComplEx(X_train, X_valid, X_test, k):
    print('Training set:', X_train.shape)

    model = ComplEx(k=k)

    model.fit(X_train)

    # create the filter （创建过滤器）
    X_filter = np.concatenate([X_train, X_valid, X_test], 0)

    ranks = evaluate_performance(X_test, 
                                 model=model,
                                 filter_triples=X_filter,
                                 corrupt_side='s,o',
                                 ranking_strategy='worst')

    # ranks are computed per triple （每三元组计算排名）
    print('Test set:', X_test.shape)
    print('Size of ranks:', ranks.shape)

    display_aggregate_metrics(ranks)
    
    return model

#### (1-1)k=5   dim=10
- 2x5

In [83]:
# 训练并评估嵌入模型
complex4_10 = embedding_by_ComplEx(X_train, X_valid, X_test, k=5)

Training set: (125074, 3)
Test set: (1000, 3)
Size of ranks: (1000, 2)
Mean Rank: 439.4365
Mean Reciprocal Rank: 0.07581114822291903
Hits@1: 0.0295
Hits@10: 0.166
Hits@100: 0.496


In [84]:
# 保存嵌入模型及嵌入向量
vector_filepath = '../data/dp_embedding_model/ComplEx/complex4_10'
model_filepath = '../data/dp_embedding_model/ComplEx/complex4_10/complex4_10.pkl'

save_embedding_vector(complex4_10, vector_filepath)
save_embedding_model(complex4_10, model_filepath)

finish!
finish!


#### (1-2)k=10   dim=20
- 4x5

In [86]:
# 训练并评估嵌入模型
complex4_20 = embedding_by_ComplEx(X_train, X_valid, X_test, k=10)

Training set: (125074, 3)
Test set: (1000, 3)
Size of ranks: (1000, 2)
Mean Rank: 419.916
Mean Reciprocal Rank: 0.0639840872419623
Hits@1: 0.022
Hits@10: 0.135
Hits@100: 0.506


In [87]:
# 保存嵌入模型及嵌入向量
vector_filepath = '../data/dp_embedding_model/ComplEx/complex4_20'
model_filepath = '../data/dp_embedding_model/ComplEx/complex4_20/complex4_20.pkl'

save_embedding_vector(complex4_20, vector_filepath)
save_embedding_model(complex4_20, model_filepath)

finish!
finish!


#### （1）k=18 dim=36
- 6x6

In [728]:
# 训练并评估嵌入模型
complex4_36 = embedding_by_ComplEx(X_train, X_valid, X_test, k=18)

Training set: (125074, 3)
Test set: (1000, 3)
Size of ranks: (1000, 2)
Mean Rank: 445.035
Mean Reciprocal Rank: 0.08191008236153474
Hits@1: 0.037
Hits@10: 0.164
Hits@100: 0.552


In [729]:
# ComplEx模型默认参数
complex4_36.all_params

{'k': 18,
 'eta': 2,
 'epochs': 100,
 'batches_count': 100,
 'seed': 0,
 'embedding_model_params': {'negative_corruption_entities': 'all',
  'corrupt_sides': ['s,o']},
 'optimizer': 'adam',
 'optimizer_params': {'lr': 0.0005},
 'loss': 'nll',
 'loss_params': {},
 'regularizer': None,
 'regularizer_params': {},
 'initializer': 'xavier',
 'initializer_params': {'uniform': False},
 'verbose': False}

In [730]:
# 保存嵌入模型及嵌入向量
vector_filepath = '../data/dp_embedding_model/ComplEx/complex4_36'
model_filepath = '../data/dp_embedding_model/ComplEx/complex4_36/complex4_36.pkl'

save_embedding_vector(complex4_36, vector_filepath)
save_embedding_model(complex4_36, model_filepath)

finish!
finish!


#### （2）k=32 dim=64
- 8x8

In [731]:
# 训练并评估嵌入模型
complex4_64 = embedding_by_ComplEx(X_train, X_valid, X_test, k=32)

Training set: (125074, 3)
Test set: (1000, 3)
Size of ranks: (1000, 2)
Mean Rank: 430.6095
Mean Reciprocal Rank: 0.09411095805023574
Hits@1: 0.0465
Hits@10: 0.177
Hits@100: 0.5975


In [732]:
# 保存嵌入模型及嵌入向量
vector_filepath = '../data/dp_embedding_model/ComplEx/complex4_64'
model_filepath = '../data/dp_embedding_model/ComplEx/complex4_64/complex4_64.pkl'

save_embedding_vector(complex4_64, vector_filepath)
save_embedding_model(complex4_64, model_filepath)

finish!
finish!


#### （3）k=98  dim=196
- 14x14

In [45]:
# 训练并评估嵌入模型
complex4_196 = embedding_by_ComplEx(X_train, X_valid, X_test, k=98)

Training set: (125074, 3)
Test set: (1000, 3)
Size of ranks: (1000, 2)
Mean Rank: 500.3905
Mean Reciprocal Rank: 0.12072138290762092
Hits@1: 0.0575
Hits@10: 0.2355
Hits@100: 0.653


In [68]:
# 保存嵌入模型及嵌入向量
vector_filepath = '../data/dp_embedding_model/ComplEx/complex4_196'
model_filepath = '../data/dp_embedding_model/ComplEx/complex4_196/complex4_196.pkl'

save_embedding_vector(complex4_196, vector_filepath)
save_embedding_model(complex4_196, model_filepath)

finish!
finish!


#### （4）k=128   dim=256
- 16x16

In [733]:
# 训练并评估嵌入模型
complex4_256 = embedding_by_ComplEx(X_train, X_valid, X_test, k=128)

Training set: (125074, 3)
Test set: (1000, 3)
Size of ranks: (1000, 2)
Mean Rank: 572.3415
Mean Reciprocal Rank: 0.12808078982507778
Hits@1: 0.0635
Hits@10: 0.2645
Hits@100: 0.664


In [734]:
# 保存嵌入模型及嵌入向量
vector_filepath = '../data/dp_embedding_model/ComplEx/complex4_256'
model_filepath = '../data/dp_embedding_model/ComplEx/complex4_256/complex4_256.pkl'

save_embedding_vector(complex4_256, vector_filepath)
save_embedding_model(complex4_256, model_filepath)

finish!
finish!


#### （4）k=162  dim=324
- 18x18

In [735]:
# 训练并评估嵌入模型
complex4_324 = embedding_by_ComplEx(X_train, X_valid, X_test, k=162)

Training set: (125074, 3)
Test set: (1000, 3)
Size of ranks: (1000, 2)
Mean Rank: 566.372
Mean Reciprocal Rank: 0.13231854904971857
Hits@1: 0.0695
Hits@10: 0.2585
Hits@100: 0.6585


In [736]:
# 保存嵌入模型及嵌入向量
vector_filepath = '../data/dp_embedding_model/ComplEx/complex4_324'
model_filepath = '../data/dp_embedding_model/ComplEx/complex4_324/complex4_324.pkl'

save_embedding_vector(complex4_324, vector_filepath)
save_embedding_model(complex4_324, model_filepath)

finish!
finish!


#### （5）k=200  dim=400
- 20x20

In [737]:
# 训练并评估嵌入模型
complex4_400 = embedding_by_ComplEx(X_train, X_valid, X_test, k=200)

Training set: (125074, 3)
Test set: (1000, 3)
Size of ranks: (1000, 2)
Mean Rank: 517.0145
Mean Reciprocal Rank: 0.1320716295801259
Hits@1: 0.0665
Hits@10: 0.2605
Hits@100: 0.6685


In [738]:
# 保存嵌入模型及嵌入向量
vector_filepath = '../data/dp_embedding_model/ComplEx/complex4_400'
model_filepath = '../data/dp_embedding_model/ComplEx/complex4_400/complex4_400.pkl'

save_embedding_vector(complex4_400, vector_filepath)
save_embedding_model(complex4_400, model_filepath)

finish!
finish!


#### （6）k=250 dim=500
- 20x25

In [739]:
# 训练并评估嵌入模型
complex4_500 = embedding_by_ComplEx(X_train, X_valid, X_test, k=250)

Training set: (125074, 3)
Test set: (1000, 3)
Size of ranks: (1000, 2)
Mean Rank: 538.4695
Mean Reciprocal Rank: 0.13860038594617508
Hits@1: 0.067
Hits@10: 0.273
Hits@100: 0.68


In [740]:
# 保存嵌入模型及嵌入向量
vector_filepath = '../data/dp_embedding_model/ComplEx/complex4_500'
model_filepath = '../data/dp_embedding_model/ComplEx/complex4_500/complex4_500.pkl'

save_embedding_vector(complex4_500, vector_filepath)
save_embedding_model(complex4_500, model_filepath)

finish!
finish!


#### （7）k=300 dim=600
- 20x30

In [741]:
# 训练并评估嵌入模型
complex4_600 = embedding_by_ComplEx(X_train, X_valid, X_test, k=300)

Training set: (125074, 3)
Test set: (1000, 3)
Size of ranks: (1000, 2)
Mean Rank: 552.615
Mean Reciprocal Rank: 0.14337369489399793
Hits@1: 0.073
Hits@10: 0.287
Hits@100: 0.679


In [742]:
# 保存嵌入模型及嵌入向量
vector_filepath = '../data/dp_embedding_model/ComplEx/complex4_600'
model_filepath = '../data/dp_embedding_model/ComplEx/complex4_600/complex4_600.pkl'

save_embedding_vector(complex4_600, vector_filepath)
save_embedding_model(complex4_600, model_filepath)

finish!
finish!


#### （8）k=350  dim=700
- 20x35

In [743]:
# 训练并评估嵌入模型
complex4_700 = embedding_by_ComplEx(X_train, X_valid, X_test, k=350)

Training set: (125074, 3)
Test set: (1000, 3)
Size of ranks: (1000, 2)
Mean Rank: 564.383
Mean Reciprocal Rank: 0.14300946035658205
Hits@1: 0.0725
Hits@10: 0.288
Hits@100: 0.682


In [744]:
# 保存嵌入模型及嵌入向量
vector_filepath = '../data/dp_embedding_model/ComplEx/complex4_700'
model_filepath = '../data/dp_embedding_model/ComplEx/complex4_700/complex4_700.pkl'

save_embedding_vector(complex4_700, vector_filepath)
save_embedding_model(complex4_700, model_filepath)

finish!
finish!


## 4.4 HolE

### 4.4.1 训练嵌入模型 

In [116]:
# 使用默认参数进行训练

def embedding_by_HolE(X_train, X_valid, X_test, k):
    print('Training set:', X_train.shape)

    model = HolE(k=k)

    model.fit(X_train)

    # create the filter （创建过滤器）
    X_filter = np.concatenate([X_train, X_valid, X_test], 0)

    ranks = evaluate_performance(X_test, 
                                 model=model,
                                 filter_triples=X_filter,
                                 corrupt_side='s,o',
                                 ranking_strategy='worst')

    # ranks are computed per triple （每三元组计算排名）
    print('Test set:', X_test.shape)
    print('Size of ranks:', ranks.shape)

    display_aggregate_metrics(ranks)
    
    return model

#### （1-1）k=5    dim=10
- 2x5

In [117]:
# 训练并评估嵌入模型
hole4_10 = embedding_by_HolE(X_train, X_valid, X_test, k=5)

Training set: (125074, 3)
Test set: (1000, 3)
Size of ranks: (1000, 2)
Mean Rank: 451.7035
Mean Reciprocal Rank: 0.08222092336442886
Hits@1: 0.0355
Hits@10: 0.1615
Hits@100: 0.4975


In [118]:
# 保存嵌入模型及嵌入向量
vector_filepath = '../data/dp_embedding_model/HolE/hole4_10'
model_filepath = '../data/dp_embedding_model/HolE/hole4_10/hole4_10.pkl'

save_embedding_vector(hole4_10, vector_filepath)
save_embedding_model(hole4_10, model_filepath)

finish!
finish!


#### （1-2）k=10    dim=20
- 4x5

In [119]:
# 训练并评估嵌入模型
hole4_20 = embedding_by_HolE(X_train, X_valid, X_test, k=10)

Training set: (125074, 3)
Test set: (1000, 3)
Size of ranks: (1000, 2)
Mean Rank: 477.574
Mean Reciprocal Rank: 0.06428134283856407
Hits@1: 0.0285
Hits@10: 0.123
Hits@100: 0.465


In [120]:
# 保存嵌入模型及嵌入向量
vector_filepath = '../data/dp_embedding_model/HolE/hole4_20'
model_filepath = '../data/dp_embedding_model/HolE/hole4_20/hole4_20.pkl'

save_embedding_vector(hole4_20, vector_filepath)
save_embedding_model(hole4_20, model_filepath)

finish!
finish!


#### （1）k=18 ——> dim=36=k*2
- 6x6

In [499]:
# 训练并评估嵌入模型
hole4_36 = embedding_by_HolE(X_train, X_valid, X_test, k=18)

Training set: (125074, 3)
Test set: (1000, 3)
Size of ranks: (1000, 2)
Mean Rank: 432.611
Mean Reciprocal Rank: 0.06940650688317013
Hits@1: 0.0295
Hits@10: 0.1325
Hits@100: 0.4805


In [500]:
# 保存嵌入模型及嵌入向量
vector_filepath = '../data/dp_embedding_model/HolE/hole4_36'
model_filepath = '../data/dp_embedding_model/HolE/hole4_36/hole4_36.pkl'

save_embedding_vector(hole4_36, vector_filepath)
save_embedding_model(hole4_36, model_filepath)

finish!
finish!


#### (2) k=32——>dim=64=k*2
- 8x8

In [519]:
# 训练并评估嵌入模型
hole4_64 = embedding_by_HolE(X_train, X_valid, X_test, k=32)

Training set: (125074, 3)
Test set: (1000, 3)
Size of ranks: (1000, 2)
Mean Rank: 424.4345
Mean Reciprocal Rank: 0.07842684654425282
Hits@1: 0.0375
Hits@10: 0.1545
Hits@100: 0.499


In [520]:
# 保存嵌入模型及嵌入向量
vector_filepath = '../data/dp_embedding_model/HolE/hole4_64'
model_filepath = '../data/dp_embedding_model/HolE/hole4_64/hole4_64.pkl'

save_embedding_vector(hole4_64, vector_filepath)
save_embedding_model(hole4_64, model_filepath)

finish!
finish!


#### (3) k=50——>dim=100=k*2
- 10x10

In [536]:
# 训练并评估嵌入模型
hole4_100 = embedding_by_HolE(X_train, X_valid, X_test, k=50)

Training set: (125074, 3)
Test set: (1000, 3)
Size of ranks: (1000, 2)
Mean Rank: 427.0415
Mean Reciprocal Rank: 0.08042671730711144
Hits@1: 0.0355
Hits@10: 0.1615
Hits@100: 0.498


In [537]:
# 保存嵌入模型及嵌入向量
vector_filepath = '../data/dp_embedding_model/HolE/hole4_100'
model_filepath = '../data/dp_embedding_model/HolE/hole4_100/hole4_100.pkl'

save_embedding_vector(hole4_100, vector_filepath)
save_embedding_model(hole4_100, model_filepath)

finish!
finish!


#### (3-1) k=64——>dim=128=k*2
- 8x16

In [805]:
# 训练并评估嵌入模型
hole4_128 = embedding_by_HolE(X_train, X_valid, X_test, k=64)

Training set: (125074, 3)
Test set: (1000, 3)
Size of ranks: (1000, 2)
Mean Rank: 422.6515
Mean Reciprocal Rank: 0.08026336178616397
Hits@1: 0.037
Hits@10: 0.1555
Hits@100: 0.5005


In [806]:
# 保存嵌入模型及嵌入向量
vector_filepath = '../data/dp_embedding_model/HolE/hole4_128'
model_filepath = '../data/dp_embedding_model/HolE/hole4_128/hole4_128.pkl'

save_embedding_vector(hole4_128, vector_filepath)
save_embedding_model(hole4_128, model_filepath)

finish!
finish!


#### (4) k=98——>dim=196=k*2
- 14x14

In [552]:
# 训练并评估嵌入模型
hole4_196 = embedding_by_HolE(X_train, X_valid, X_test, k=98)

Training set: (125074, 3)
Test set: (1000, 3)
Size of ranks: (1000, 2)
Mean Rank: 410.8525
Mean Reciprocal Rank: 0.08627848988890921
Hits@1: 0.0435
Hits@10: 0.1695
Hits@100: 0.506


In [554]:
# 保存嵌入模型及嵌入向量
vector_filepath = '../data/dp_embedding_model/HolE/hole4_196'
model_filepath = '../data/dp_embedding_model/HolE/hole4_196/hole4_196.pkl'

save_embedding_vector(hole4_196, vector_filepath)
save_embedding_model(hole4_196, model_filepath)

finish!
finish!


#### (5) k=128——>dim=256=k*2
- 16x16

In [553]:
# 训练并评估嵌入模型
hole4_256 = embedding_by_HolE(X_train, X_valid, X_test, k=128)

Training set: (125074, 3)
Test set: (1000, 3)
Size of ranks: (1000, 2)
Mean Rank: 415.3785
Mean Reciprocal Rank: 0.0823447940341189
Hits@1: 0.039
Hits@10: 0.156
Hits@100: 0.5065


In [569]:
# 保存嵌入模型及嵌入向量
vector_filepath = '../data/dp_embedding_model/HolE/hole4_256'
model_filepath = '../data/dp_embedding_model/HolE/hole4_256/hole4_256.pkl'

save_embedding_vector(hole4_256, vector_filepath)
save_embedding_model(hole4_256, model_filepath)

finish!
finish!


#### （6）k=162——>dim=324=2*k
- 18x18

In [584]:
# 训练并评估嵌入模型
hole4_324 = embedding_by_HolE(X_train, X_valid, X_test, k=162)

Training set: (125074, 3)
Test set: (1000, 3)
Size of ranks: (1000, 2)
Mean Rank: 408.9525
Mean Reciprocal Rank: 0.08288156187665563
Hits@1: 0.04
Hits@10: 0.162
Hits@100: 0.5055


In [585]:
# 保存嵌入模型及嵌入向量
vector_filepath = '../data/dp_embedding_model/HolE/hole4_324'
model_filepath = '../data/dp_embedding_model/HolE/hole4_324/hole4_324.pkl'

save_embedding_vector(hole4_324, vector_filepath)
save_embedding_model(hole4_324, model_filepath)

finish!
finish!


#### （6）k=200——>dim=400=2*k
- 20x20

In [600]:
# 训练并评估嵌入模型
hole4_400 = embedding_by_HolE(X_train, X_valid, X_test, k=200)

Training set: (125074, 3)
Test set: (1000, 3)
Size of ranks: (1000, 2)
Mean Rank: 409.1155
Mean Reciprocal Rank: 0.08337226327263067
Hits@1: 0.04
Hits@10: 0.1595
Hits@100: 0.5025


In [601]:
# 保存嵌入模型及嵌入向量
vector_filepath = '../data/dp_embedding_model/HolE/hole4_400'
model_filepath = '../data/dp_embedding_model/HolE/hole4_400/hole4_400.pkl'

save_embedding_vector(hole4_400, vector_filepath)
save_embedding_model(hole4_400, model_filepath)

finish!
finish!


#### (7)k=250  dim=500
- 20x25

In [93]:
# 训练并评估嵌入模型
hole4_500 = embedding_by_HolE(X_train, X_valid, X_test, k=250)

Training set: (125074, 3)
Test set: (1000, 3)
Size of ranks: (1000, 2)
Mean Rank: 402.7735
Mean Reciprocal Rank: 0.08280486069168615
Hits@1: 0.0375
Hits@10: 0.1615
Hits@100: 0.5075


In [94]:
# 保存嵌入模型及嵌入向量
vector_filepath = '../data/dp_embedding_model/HolE/hole4_500'
model_filepath = '../data/dp_embedding_model/HolE/hole4_500/hole4_500.pkl'

save_embedding_vector(hole4_500, vector_filepath)
save_embedding_model(hole4_500, model_filepath)

finish!
finish!


#### (8) k=300   dim=600
- 20x30

In [108]:
# 训练并评估嵌入模型
hole4_600 = embedding_by_HolE(X_train, X_valid, X_test, k=300)

Training set: (125074, 3)
Test set: (1000, 3)
Size of ranks: (1000, 2)
Mean Rank: 401.3255
Mean Reciprocal Rank: 0.08523958677829532
Hits@1: 0.0405
Hits@10: 0.1645
Hits@100: 0.5035


In [109]:
# 保存嵌入模型及嵌入向量
vector_filepath = '../data/dp_embedding_model/HolE/hole4_600'
model_filepath = '../data/dp_embedding_model/HolE/hole4_600/hole4_600.pkl'

save_embedding_vector(hole4_600, vector_filepath)
save_embedding_model(hole4_600, model_filepath)

finish!
finish!


#### (9)k=350   dim=700
- 20x35

In [123]:
# 训练并评估嵌入模型
hole4_700 = embedding_by_HolE(X_train, X_valid, X_test, k=350)

Training set: (125074, 3)
Test set: (1000, 3)
Size of ranks: (1000, 2)
Mean Rank: 408.841
Mean Reciprocal Rank: 0.08167508807384541
Hits@1: 0.037
Hits@10: 0.156
Hits@100: 0.4985


In [124]:
# 保存嵌入模型及嵌入向量
vector_filepath = '../data/dp_embedding_model/HolE/hole4_700'
model_filepath = '../data/dp_embedding_model/HolE/hole4_700/hole4_700.pkl'

save_embedding_vector(hole4_700, vector_filepath)
save_embedding_model(hole4_700, model_filepath)

finish!
finish!


#### (10)k=400   dim=800
- 20x40

In [125]:
# 训练并评估嵌入模型
hole4_800 = embedding_by_HolE(X_train, X_valid, X_test, k=400)

Training set: (125074, 3)
Test set: (1000, 3)
Size of ranks: (1000, 2)
Mean Rank: 399.263
Mean Reciprocal Rank: 0.08561201547891323
Hits@1: 0.04
Hits@10: 0.166
Hits@100: 0.5025


In [126]:
# 保存嵌入模型及嵌入向量
vector_filepath = '../data/dp_embedding_model/HolE/hole4_800'
model_filepath = '../data/dp_embedding_model/HolE/hole4_800/hole4_800.pkl'

save_embedding_vector(hole4_800, vector_filepath)
save_embedding_model(hole4_800, model_filepath)

finish!
finish!


## 4.5 TransE

### 4.5.1 训练嵌入模型 

In [153]:
# 使用默认参数进行训练

def embedding_by_TransE(X_train, X_valid, X_test, k):
    print('Training set:', X_train.shape)

    model = TransE(k=k)

    model.fit(X_train)

    # create the filter （创建过滤器）
    X_filter = np.concatenate([X_train, X_valid, X_test], 0)

    ranks = evaluate_performance(X_test, 
                                 model=model,
                                 filter_triples=X_filter,
                                 corrupt_side='s,o',
                                 ranking_strategy='worst')

    # ranks are computed per triple （每三元组计算排名）
    print('Test set:', X_test.shape)
    print('Size of ranks:', ranks.shape)

    display_aggregate_metrics(ranks)
    
    return model

#### （1-1）k=10
- 2x5

In [154]:
# 训练并评估嵌入模型
transe4_10 = embedding_by_TransE(X_train, X_valid, X_test, k=10)

Training set: (125074, 3)
Test set: (1000, 3)
Size of ranks: (1000, 2)
Mean Rank: 501.7095
Mean Reciprocal Rank: 0.031037821823515742
Hits@1: 0.0135
Hits@10: 0.048
Hits@100: 0.283


In [155]:
# 保存嵌入模型及嵌入向量
vector_filepath = '../data/dp_embedding_model/TransE/transe4_10'
model_filepath = '../data/dp_embedding_model/TransE/transe4_10/transe4_10.pkl'

save_embedding_vector(transe4_10, vector_filepath)
save_embedding_model(transe4_10, model_filepath)

finish!
finish!


#### （1-2）k=20
- 4x5

In [156]:
# 训练并评估嵌入模型
transe4_20 = embedding_by_TransE(X_train, X_valid, X_test, k=20)

Training set: (125074, 3)
Test set: (1000, 3)
Size of ranks: (1000, 2)
Mean Rank: 473.2895
Mean Reciprocal Rank: 0.03303283718318699
Hits@1: 0.012
Hits@10: 0.058
Hits@100: 0.3135


In [157]:
# 保存嵌入模型及嵌入向量
vector_filepath = '../data/dp_embedding_model/TransE/transe4_20'
model_filepath = '../data/dp_embedding_model/TransE/transe4_20/transe4_20.pkl'

save_embedding_vector(transe4_20, vector_filepath)
save_embedding_model(transe4_20, model_filepath)

finish!
finish!


#### （1）k=36
- 6x6

In [199]:
# 训练并评估嵌入模型
transe4_36 = embedding_by_TransE(X_train, X_valid, X_test, k=36)

Training set: (125074, 3)
Test set: (1000, 3)
Size of ranks: (1000, 2)
Mean Rank: 451.81
Mean Reciprocal Rank: 0.04040993785184323
Hits@1: 0.0165
Hits@10: 0.071
Hits@100: 0.34


In [200]:
transe4_36.all_params

{'k': 36,
 'eta': 2,
 'epochs': 100,
 'batches_count': 100,
 'seed': 0,
 'embedding_model_params': {'norm': 1,
  'normalize_ent_emb': False,
  'negative_corruption_entities': 'all',
  'corrupt_sides': ['s,o']},
 'optimizer': 'adam',
 'optimizer_params': {'lr': 0.0005},
 'loss': 'nll',
 'loss_params': {},
 'regularizer': None,
 'regularizer_params': {},
 'initializer': 'xavier',
 'initializer_params': {'uniform': False},
 'verbose': False}

In [201]:
# 保存嵌入模型及嵌入向量
vector_filepath = '../data/dp_embedding_model/TransE/transe4_36'
model_filepath = '../data/dp_embedding_model/TransE/transe4_36/transe4_36.pkl'

save_embedding_vector(transe4_36, vector_filepath)
save_embedding_model(transe4_36, model_filepath)

finish!
finish!


#### （2）k=64
- 8x8

In [219]:
# 训练并评估嵌入模型
transe4_64 = embedding_by_TransE(X_train, X_valid, X_test, k=64)

Training set: (125074, 3)
Test set: (1000, 3)
Size of ranks: (1000, 2)
Mean Rank: 445.6645
Mean Reciprocal Rank: 0.045471426366908554
Hits@1: 0.0205
Hits@10: 0.088
Hits@100: 0.3705


In [220]:
# 保存嵌入模型及嵌入向量
vector_filepath = '../data/dp_embedding_model/TransE/transe4_64'
model_filepath = '../data/dp_embedding_model/TransE/transe4_64/transe4_64.pkl'

save_embedding_vector(transe4_64, vector_filepath)
save_embedding_model(transe4_64, model_filepath)

finish!
finish!


#### （3）k=121
- 11x11

In [238]:
# 训练并评估嵌入模型
transe4_121 = embedding_by_TransE(X_train, X_valid, X_test, k=121)

Training set: (125074, 3)
Test set: (1000, 3)
Size of ranks: (1000, 2)
Mean Rank: 435.337
Mean Reciprocal Rank: 0.052450077543397526
Hits@1: 0.0215
Hits@10: 0.101
Hits@100: 0.3985


In [239]:
# 保存嵌入模型及嵌入向量
vector_filepath = '../data/dp_embedding_model/TransE/transe4_121'
model_filepath = '../data/dp_embedding_model/TransE/transe4_121/transe4_121.pkl'

save_embedding_vector(transe4_121, vector_filepath)
save_embedding_model(transe4_121, model_filepath)

finish!
finish!


#### （3-1）k=128
- 8x16

In [807]:
# 训练并评估嵌入模型
transe4_128 = embedding_by_TransE(X_train, X_valid, X_test, k=128)

Training set: (125074, 3)
Test set: (1000, 3)
Size of ranks: (1000, 2)
Mean Rank: 434.53
Mean Reciprocal Rank: 0.055754547590485884
Hits@1: 0.0265
Hits@10: 0.1065
Hits@100: 0.4


In [808]:
# 保存嵌入模型及嵌入向量
vector_filepath = '../data/dp_embedding_model/TransE/transe4_128'
model_filepath = '../data/dp_embedding_model/TransE/transe4_128/transe4_128.pkl'

save_embedding_vector(transe4_128, vector_filepath)
save_embedding_model(transe4_128, model_filepath)

finish!
finish!


#### （3-2）k=196
- 14x14

In [48]:
# 训练并评估嵌入模型
transe4_196 = embedding_by_TransE(X_train, X_valid, X_test, k=196)

Training set: (125074, 3)
Test set: (1000, 3)
Size of ranks: (1000, 2)
Mean Rank: 433.5845
Mean Reciprocal Rank: 0.05711183525494771
Hits@1: 0.0265
Hits@10: 0.106
Hits@100: 0.4155


In [84]:
# 保存嵌入模型及嵌入向量
vector_filepath = '../data/dp_embedding_model/TransE/transe4_196'
model_filepath = '../data/dp_embedding_model/TransE/transe4_196/transe4_196.pkl'

save_embedding_vector(transe4_196, vector_filepath)
save_embedding_model(transe4_196, model_filepath)

finish!
finish!


#### （4）k=256
- 16x16

In [254]:
# 训练并评估嵌入模型
transe4_256 = embedding_by_TransE(X_train, X_valid, X_test, k=256)

Training set: (125074, 3)
Test set: (1000, 3)
Size of ranks: (1000, 2)
Mean Rank: 439.609
Mean Reciprocal Rank: 0.05953972926263759
Hits@1: 0.027
Hits@10: 0.1155
Hits@100: 0.409


In [255]:
# 保存嵌入模型及嵌入向量
vector_filepath = '../data/dp_embedding_model/TransE/transe4_256'
model_filepath = '../data/dp_embedding_model/TransE/transe4_256/transe4_256.pkl'

save_embedding_vector(transe4_256, vector_filepath)
save_embedding_model(transe4_256, model_filepath)

finish!
finish!


#### （5）k=324
- 18x18

In [271]:
# 训练并评估嵌入模型
transe4_324 = embedding_by_TransE(X_train, X_valid, X_test, k=324)

Training set: (125074, 3)
Test set: (1000, 3)
Size of ranks: (1000, 2)
Mean Rank: 439.484
Mean Reciprocal Rank: 0.0564043277743103
Hits@1: 0.026
Hits@10: 0.1095
Hits@100: 0.4165


In [272]:
# 保存嵌入模型及嵌入向量
vector_filepath = '../data/dp_embedding_model/TransE/transe4_324'
model_filepath = '../data/dp_embedding_model/TransE/transe4_324/transe4_324.pkl'

save_embedding_vector(transe4_324, vector_filepath)
save_embedding_model(transe4_324, model_filepath)

finish!
finish!


#### （5）k=400
- 20x20

In [287]:
# 训练并评估嵌入模型
transe4_400 = embedding_by_TransE(X_train, X_valid, X_test, k=400)

Training set: (125074, 3)
Test set: (1000, 3)
Size of ranks: (1000, 2)
Mean Rank: 443.987
Mean Reciprocal Rank: 0.05555089812106204
Hits@1: 0.021
Hits@10: 0.1085
Hits@100: 0.421


In [288]:
# 保存嵌入模型及嵌入向量
vector_filepath = '../data/dp_embedding_model/TransE/transe4_400'
model_filepath = '../data/dp_embedding_model/TransE/transe4_400/transe4_400.pkl'

save_embedding_vector(transe4_400, vector_filepath)
save_embedding_model(transe4_400, model_filepath)

finish!
finish!


#### （6）k=500
- 20x25

In [303]:
# 训练并评估嵌入模型
transe4_500 = embedding_by_TransE(X_train, X_valid, X_test, k=500)

Training set: (125074, 3)
Test set: (1000, 3)
Size of ranks: (1000, 2)
Mean Rank: 444.711
Mean Reciprocal Rank: 0.05507489243100148
Hits@1: 0.0215
Hits@10: 0.1095
Hits@100: 0.4325


In [304]:
# 保存嵌入模型及嵌入向量
vector_filepath = '../data/dp_embedding_model/TransE/transe4_500'
model_filepath = '../data/dp_embedding_model/TransE/transe4_500/transe4_500.pkl'

save_embedding_vector(transe4_500, vector_filepath)
save_embedding_model(transe4_500, model_filepath)

finish!
finish!


#### （7）k=600
- 20x30

In [321]:
# 训练并评估嵌入模型
transe4_600 = embedding_by_TransE(X_train, X_valid, X_test, k=600)

Training set: (125074, 3)
Test set: (1000, 3)
Size of ranks: (1000, 2)
Mean Rank: 446.213
Mean Reciprocal Rank: 0.05393715974879954
Hits@1: 0.0225
Hits@10: 0.108
Hits@100: 0.4325


In [322]:
# 保存嵌入模型及嵌入向量
vector_filepath = '../data/dp_embedding_model/TransE/transe4_600'
model_filepath = '../data/dp_embedding_model/TransE/transe4_600/transe4_600.pkl'

save_embedding_vector(transe4_600, vector_filepath)
save_embedding_model(transe4_600, model_filepath)

finish!
finish!


#### （8）k=700
- 20x35

In [669]:
# 训练并评估嵌入模型
transe4_700 = embedding_by_TransE(X_train, X_valid, X_test, k=700)

Training set: (125074, 3)
Test set: (1000, 3)
Size of ranks: (1000, 2)
Mean Rank: 447.823
Mean Reciprocal Rank: 0.04918662576823776
Hits@1: 0.0165
Hits@10: 0.097
Hits@100: 0.417


In [670]:
# 保存嵌入模型及嵌入向量
vector_filepath = '../data/dp_embedding_model/TransE/transe4_700'
model_filepath = '../data/dp_embedding_model/TransE/transe4_700/transe4_700.pkl'

save_embedding_vector(transe4_700, vector_filepath)
save_embedding_model(transe4_700, model_filepath)

finish!
finish!


#### （9）k=800
- 20x40

In [685]:
# 训练并评估嵌入模型
transe4_800 = embedding_by_TransE(X_train, X_valid, X_test, k=800)

Training set: (125074, 3)
Test set: (1000, 3)
Size of ranks: (1000, 2)
Mean Rank: 445.3725
Mean Reciprocal Rank: 0.05355590520123877
Hits@1: 0.0215
Hits@10: 0.1045
Hits@100: 0.4175


In [686]:
# 保存嵌入模型及嵌入向量
vector_filepath = '../data/dp_embedding_model/TransE/transe4_800'
model_filepath = '../data/dp_embedding_model/TransE/transe4_800/transe4_800.pkl'

save_embedding_vector(transe4_800, vector_filepath)
save_embedding_model(transe4_800, model_filepath)

finish!
finish!


# 5 通过嵌入向量构造训练集和测试集的样本特征数据

In [1]:
import pandas as pd
import pickle

## 5.1 构建实体向量字典

### 5.1.1 导入实体名称

In [176]:
metadata = pd.read_table('../data/dp_embedding_model/TransE/transe4_20/metadata.tsv', header=None)
metadata    # 实体名称

0
0      A0A023W3H0
1      A0A068JFB7
2      A0A0E1R3H3
3      A0A0F7R7F9
4      A0A0H2UQB5
...           ...
18222      Q9ZIV5
18223      Q9ZK53
18224      Q9ZLT0
18225      Q9ZNA2
18226      T1RTG8

[18227 rows x 1 columns]

### 5.1.2 导入实体向量

In [177]:
embedding_vetor = pd.read_table('../data/dp_embedding_model/TransE/transe4_20/embeddings_projector.tsv', header=None)
embedding_vetor.head()

0         1         2         3         4         5         6   \
0 -0.039842  0.176389  0.096765  0.120724 -0.117471  0.121969  0.144579   
1 -0.043593  0.173400  0.092968  0.122657 -0.109141  0.122883  0.150502   
2 -0.181569  0.423642 -0.041805 -0.016747  0.035369 -0.112371  0.375543   
3  0.333842  0.069378 -0.011116  0.096536 -0.034945 -0.014934  0.008229   
4  0.328485  0.140945  0.037698  0.011169 -0.003478  0.084070  0.037869   

         7         8         9         10        11        12        13  \
0  0.072815  0.258944  0.101939 -0.085492 -0.005261  0.017183 -0.057614   
1  0.072614  0.263450  0.096609 -0.088787 -0.011013  0.009639 -0.048875   
2 -0.026266  0.143770  0.289096  0.032639  0.199942 -0.253892  0.086222   
3  0.105026  0.226993 -0.055133 -0.089356  0.087693 -0.455903 -0.057248   
4  0.002576  0.373048  0.112381  0.029601 -0.055654 -0.087605 -0.067504   

         14        15        16        17        18        19  
0  0.158313 -0.079483 -0.259324 -0.300754  0.137414 -0.296191  
1  0.164267 -0.077079 -0.253318 -0.306054  0.138425 -0.294399  
2  0.461760  0.160967 -0.343145 -0.107320  0.280999 -0.154945  
3  0.338787 -0.069623 -0.004359  0.144517  0.040207 -0.011370  
4  0.246870 -0.100112 -0.242472 -0.368891  0.009356  0.014619

In [178]:
embedding_vetor.shape

(18227, 20)

### 5.1.3 构建实体-向量键值对字典

In [179]:
# 构建实体-向量键值对字典
entity_vector_dict = {}

for i in range(len(embedding_vetor)):
    entity_vector_dict[metadata.iloc[i,0]] = list(embedding_vetor.iloc[i,:])

In [180]:
len(entity_vector_dict)

18227

In [181]:
# 保存entity-vector字典
with open('../data/dp_embedding_model/TransE/transe4_20/entity_vector_dict4_10.pkl', 'wb') as f:
    pickle.dump(entity_vector_dict, f)

## 5.2 构建数据集的实体-向量（entity-vector）表示矩阵

In [182]:
# 训练集
train_dataset

subject    object  label
0       DB00001  C0002965      0
1       DB00001  C0002965      0
2       DB00001  C0002965      0
3       DB00001  C0002965      0
4       DB00001  C0002965      0
...         ...       ...    ...
190594  DB15263  C0376288      1
190595  DB15263  C0541799      1
190596  DB15263  C1268766      1
190597  DB15263  C2225524      1
190598  DB15263  C2242996      1

[190599 rows x 3 columns]

In [183]:
# 测试集
test_dataset

subject    object  label
0     DB00513  C0023890      0
1     DB01320  C0011206      0
2     DB01241  C0030305      0
3     DB01141  C0239295      0
4     DB01059  C0033581      0
...       ...       ...    ...
2023  DB00776  C0043352      1
2024  DB00401  C0423791      1
2025  DB01259  C0030252      1
2026  DB00526  C0178298      1
2027  DB01037  C0231218      1

[2028 rows x 3 columns]

### 5.2.1 构建训练集和测试集的特征向量矩阵
- 使用头实体向量 - 尾实体向量的方式进行构造

In [184]:
list(train_dataset.iloc[0])

['DB00001', 'C0002965', 0]

In [130]:
def entity_dataset_to_vector(dataset, entity_vector_dict):
    feature_data_list = []

    for i in range(len(dataset)):
        row_data = list(dataset.iloc[i])
        subject_key = row_data[0]
        object_key = row_data[1]
        subject_value = entity_vector_dict[subject_key]
        object_value = entity_vector_dict[object_key]
        subject_sub_object = list(map(lambda x: x[0]-x[1], zip(subject_value, object_value)))   # 将来个实体向量相减
        label_value = row_data[2]

        feature_data = [subject_key] + [object_key] + subject_sub_object + [label_value]
        feature_data_list.append(feature_data)
    
    data_matrix = pd.DataFrame(feature_data_list)
    return data_matrix

In [185]:
# 训练集特征向量矩阵
train_data_matrix = entity_dataset_to_vector(train_dataset, entity_vector_dict)

# 测试集特征向量矩阵
test_data_matrix = entity_dataset_to_vector(test_dataset, entity_vector_dict)

In [186]:
test_data_matrix.head()

0         1         2         3         4         5         6   \
0  DB00513  C0023890 -0.206473  0.452072 -0.385450 -0.348839  0.350647   
1  DB01320  C0011206 -0.206597  0.452055 -0.386671 -0.349744  0.350468   
2  DB01241  C0030305 -0.207248  0.452102 -0.386593 -0.350376  0.351875   
3  DB01141  C0239295 -0.209139  0.452807 -0.387407 -0.351177  0.351527   
4  DB01059  C0033581 -0.206361  0.450089 -0.386178 -0.349862  0.349613   

         7         8         9   ...        13        14        15        16  \
0 -0.442941  0.476418 -0.327763  ...  0.288815 -0.046437  0.300915 -0.026809   
1 -0.442827  0.475666 -0.328063  ...  0.290039 -0.047146  0.301873 -0.027320   
2 -0.442568  0.476776 -0.328303  ...  0.290323 -0.047641  0.302033 -0.026305   
3 -0.442796  0.477169 -0.327581  ...  0.289308 -0.048778  0.302818 -0.027019   
4 -0.442442  0.475706 -0.327263  ...  0.289058 -0.046694  0.301435 -0.026848   

         17        18        19        20        21  22  
0  0.446413 -0.227097  0.196004  0.356313  0.263151   0  
1  0.446305 -0.227204  0.195839  0.357184  0.263942   0  
2  0.446690 -0.227123  0.195236  0.358087  0.263333   0  
3  0.446027 -0.227085  0.196295  0.358811  0.265809   0  
4  0.445685 -0.227178  0.195860  0.357180  0.262982   0  

[5 rows x 23 columns]

### 5.2.2 分离样本特征数据和标签数据

In [187]:
# 打乱数据集顺序
from sklearn.utils import shuffle
train_data_matrix = shuffle(train_data_matrix, random_state=42)
test_data_matrix = shuffle(test_data_matrix, random_state=42)

In [188]:
# 训练集特征数据及标签
train_data = train_data_matrix.iloc[:, 2:-1].values
train_label = train_data_matrix.iloc[:,-1].values

# 测试集特征数据及标签
test_data = test_data_matrix.iloc[:, 2:-1].values
test_label = test_data_matrix.iloc[:,-1].values

In [189]:
# 构造训练样本和测试样本字典
train_test_dataset = {'train_data':train_data, 'train_label':train_label, 'test_data':test_data, 'test_label':test_label}

### 5.2.3 保存数据集向量矩阵

In [190]:
# 保存数据
with open('../data/dp_embedding_model/TransE/transe4_20/train_test_dataset.pkl', 'wb') as f:
    pickle.dump(train_test_dataset, f)

print('保存成功！')

保存成功！


In [192]:
# 检查保存的数据
with open('../data/dp_embedding_model/TransE/transe4_20/train_test_dataset.pkl', 'rb') as f:
    d = pickle.load(f)

In [193]:
len(d['train_data'][0])
# d

20